In [1]:
from ipynb.fs.full import models
from ipynb.fs.full import data as dt

cuda
13908


In [2]:
import os
import cv2
import math
import torch
from owlready2 import *
import pandas as pd
from cvzone.HandTrackingModule import HandDetector
from torchvision import transforms
import numpy as np
import mediapipe as mp

2023-06-21 20:11:23.987841: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-21 20:11:25.465944: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/soumen/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2023-06-21 20:11:25.466146: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/soumen/anaconda3/lib/python3.9/site-packages/cv2

In [3]:
#Global variable declaration
root_dir = 'Mudra Dataset/Single hand'
onto_filepath = 'mudraOntology.owl'
onto = get_ontology(onto_filepath).load()
fingerDict = {1:"Thumb", 5:"Index", 9:"Middle", 13:"Ring", 17:"Little"}
annotationPath = "Annotation"

* Owlready2 * WARNING: DataProperty http://www.semanticweb.org/soumen/ontologies/2023/4/mudra-ontology#hasCoordinate belongs to more than one entity types: [owl.ObjectProperty, owl.DatatypeProperty]; I'm trying to fix it...


In [4]:
#Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
print(device)

cuda


In [18]:
class handTracker():
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5, modelComplexity=1, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.modelComplex = modelComplexity
        self.trackCon = trackCon
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mpHands.Hands(self.mode, self.maxHands, self.modelComplex, self.detectionCon, self.trackCon))
        self.mpDraw = mp.solutions.drawing_utils
    
    def handsFinder(self, image, draw=True):
        imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imageRGB)

        #if self.results.multi_hand_landmarks:
            #for handLms in self.results.multi_hand_landmarks:
                #print(handLms)
        return image
    
    def positionFinder(self, image, handNo=0, draw=True):
        lmlist=[]
        if self.results.multi_hand_landmarks:
            Hand = self.results.multi_hand_landmarks[handNo]

            for id, lm in enumerate(Hand.landmark):
                h,w,c = image.shape
                cx, cy = int(lm.x*w), int(lm.y*h)
                lmlist.append([id,cx,cy])
        return lmlist

In [6]:
data = {}
if os.path.isdir(root_dir):
    for dirs in os.listdir(root_dir):
        full_path = os.path.join(root_dir, dirs)
        csv_path = os.path.join(full_path, 'Annotation.csv')
        csv_file = pd.read_csv(csv_path)
        for i, row in csv_file.iterrows():
            img_path = os.path.join(full_path, row[0])
            ontoFileName = dirs + "_" + row[0].split(".")[0]
            #print(ontoFileName)
            image = cv2.imread(img_path)
            data[ontoFileName] = image

In [7]:
def compute_joint_angle(data, a, b, c, d, ontoFileName):
    jointAngle = []
    dr1 = [data[a][0] - data[0][0], data[a][1] - data[0][1], data[a][2] - data[0][2]]
    dr2 = [data[b][0] - data[a][0], data[b][1] - data[a][1], data[b][2] - data[a][2]]
    dr3 = [data[c][0] - data[b][0], data[c][1] - data[b][1], data[c][2] - data[b][2]]
    dr4 = [data[d][0] - data[c][0], data[d][1] - data[c][1], data[d][2] - data[c][2]]
    
    m1 = math.sqrt(dr1[0] * dr1[0] + dr1[1] * dr1[1] + dr1[2] * dr1[2])
    m2 = math.sqrt(dr2[0] * dr2[0] + dr2[1] * dr2[1] + dr2[2] * dr2[2])
    DIPAngle = math.degrees(math.acos((dr1[0] * dr2[0] + dr1[1] * dr2[1] + dr1[2] * dr2[2])/(m1*m2)))
    
    m1 = m2
    m2 = math.sqrt(dr3[0] * dr3[0] + dr3[1] * dr3[1] + dr3[2] * dr3[2])
    PIPAngle = math.degrees(math.acos((dr2[0] * dr3[0] + dr2[1] * dr3[1] + dr2[2] * dr3[2])/(m1*m2)))
    
    m1 = m2
    m2 = math.sqrt(dr4[0] * dr4[0] + dr4[1] * dr4[1] + dr4[2] * dr4[2])
    MCPAngle = math.degrees(math.acos((dr3[0] * dr4[0] + dr3[1] * dr4[1] + dr3[2] * dr4[2])/(m1*m2)))
    
    for finger in onto.Finger.instances():
        if a == 1 and finger.name == fingerDict[1]:
            for i in finger.hasJoints:
                if i.name == "IP":
                    i.hasFlexionAngle = DIPAngle
                elif i.name == "MCP":
                    i.hasFlexionAngle = PIPAngle
                elif i.name == "CMC":
                    i.hasFlexionAngle = MCPAngle
        elif finger.name == fingerDict[a]:
            for i in finger.hasJoints:
                if i.name == "DIP":
                    i.hasFlexionAngle = DIPAngle
                elif i.name == "PIP":
                    i.hasFlexionAngle = PIPAngle
                elif i.name == "MCP":
                    i.hasFlexionAngle = MCPAngle
    
    return jointAngle

In [8]:
detector = HandDetector(maxHands=1, detectionCon=0.4)
touchModel = models.AttnVGG(num_classes=160, normalize_attn=True)
touchModel.load_state_dict(torch.load("/home/soumen/Soumen/2nd/Touch model/touch_model_final.pth"))

/home/soumen/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/soumen/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


<All keys matched successfully>

In [14]:
def populateTouch(image, key):
    transform = transforms.Compose([dt.Rescale((227,227)), dt.ToTensor()])
    #print(image.type)
    sample = {'image':image.astype('float'), 'label':np.zeros([160]), 'name': key}
    res = dt.Rescale((227,227))
    trans = dt.ToTensor()
    sample = res(sample)
    sample = trans(sample)
    image = sample['image']
    #image = image.to(device)
    #print(image.shape[0])
    image = image.reshape([1, image.shape[0], image.shape[1], image.shape[2]])
    #model = touchModel.to(device)
    #print(image.shape)
    out,_,_ = touchModel(image)
    
    print(out)
    for i in range(len(out)):
        if out[i].round() > 0:
            print(key, i)
            

In [22]:
if not os.path.isdir(annotationPath):
    os.mkdir(annotationPath)
m = 0
n = 0
tracker = handTracker()
for key in data:
    m += 1
    image = data[key]
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #hands, img = detector.findHands(image)
    image = tracker.handsFinder(image)
    lmlist = tracker.positionFinder(image)
    #print(len(lmlist))
    if len(lmlist)>0:
        n+=1
    """
    result = []
    m += 1
    if hands:
        n += 1
        hand = hands[0]
        lmlist = hand['lmList']
        
        index_MCP = lmlist[5]
        little_MCP = lmlist[17]
        centre = [(index_MCP[0]+little_MCP[0])/2, (index_MCP[1]+little_MCP[1])/2, (index_MCP[2]+little_MCP[2])/2]
        for lm in lmlist:
            result.append([lm[0]-centre[0], lm[1]-centre[1], lm[2]-centre[2]])
        
        ontoFile = "Annotation_" + key
        #Calculate joint angles for each finger joint
        thumbA = compute_joint_angle(result,1,2,3,4,ontoFile)
        indexA = compute_joint_angle(result,5,6,7,8,ontoFile)
        middleA = compute_joint_angle(result,9,10,11,12,ontoFile)
        ringA = compute_joint_angle(result,13,14,15,16,ontoFile)
        littleA = compute_joint_angle(result,17,18,19,20,ontoFile)
        
        
        
        #Constructing the annotation based on created ontology for each image
        fullPath = os.path.join(annotationPath, ontoFile)
        onto.save(file = fullPath, format="rdfxml")
    """
print(n,m)


10172 13908


In [19]:
all_dataset = dt.TouchDataset(root_dir='Mudra Dataset/Single hand', num_classes=160,
                                  transform=transforms.Compose([dt.Rescale((227,227)), dt.ToTensor()]))
loader = dt.data_loader(all_dataset, batch_size=16, test=True)

In [52]:
for i, sample in enumerate(loader):
    image = sample['image']
    name = sample['name']
    out,_,_ = touchModel(image)
    o = out[0].detach().numpy()
    output = np.empty((16,160))
    for i in range(len(out)):
        o = out[i].detach().numpy()
        output = np.append(output, np.array(o), axis=1)
        #output = np.vstack((output, o))
    output = np.delete(output, slice(0,160), axis=1)
    #print(output.shape[0])
    for i in range(output.shape[0]):
        fname = name[i]
        fname = fname.split('.')[0]
        fname = fname.split('/')
        x = 'Annotation_'+fname[-2]+'_'+fname[-1]
        filename = os.path.join('Annotation', x)
        onto = get_ontology(filename).load()
        print(onto.Finger.name)
        o_arr = output[i,:].round()
        for j in range(len(x)):
            if o_arr[0] == 1:
                break
            elif o_arr[j] == 1:
                if j <= 64:
                    x = j/16 + 1
                    y = j%16

Finger
Finger
Finger


FileNotFoundError: [Errno 2] No such file or directory: 'Annotation/Annotation_Mrigashirsha_136'